In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#定义Xaiver初始化器
def xavier_init(fan_in,fan_out,constant=1):
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = -constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in,fan_out),
                            minval=low,maxval=high,
                            dtype=tf.float32)

In [14]:
#定义去噪自编码的class
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self,n_input,n_hidden,transfer_function=tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(),scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_wights = self._initialize_weights()
        self.weights = network_wights
        #定义网络结构
        self.x = tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + scale * tf.random_normal((n_input,)),
                                                    self.weights['w1']),self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,self.weights['w2']),self.weights['b2'])
        #定义自编码器的损失函数
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    #定义初始化权重的函数
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input,self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input],dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype = tf.float32))
        return all_weights
    #定义损失cost及一步训练的函数
    def partial_fit(self,X):
        cost,opt = self.sess.run((self.cost,self.optimizer),
                                feed_dict={self.x:X,self.scale:self.training_scale})
        return cost
    #定义一个总体性能评价的cost
    def calc_total_cost(self,X):
        return self.sess.run(self.cost,feed_dict = {self.x:X,
                                                   self.scale:self.training_scale})
    #定义函数，返回自编码器隐含层的输出结果
    def transform(self,X):
        return self.sess.run(self.hidden,feed_dict={self.x:X,
                                                   self.scale:slef.training_scale})
    #定义函数，它将隐含层的输出结果作为输入，通过之后的重建层将提取到的高阶特征复原为原始数据
    def generate(self,hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights['b1'])
        return self.sess.run(self.reconstruction,feed_dict={self.hidden:hidden})
    #定义函数，它整体运行一遍复原过程，包括提取高阶特征和通过高阶特征复原数据
    def reconstruct(self,X):
        return self.sess.run(self.reconstruction,feed_dict={self.x:X,self.scale:self.training_scale})
    #获取w1的权重
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    #获取偏置系数b1
    def getBiases(self):
        return self.sess.run(self.weights['b1'])

In [5]:
#使用定义好的AGN自编码器在MNIST数据集上进行一些简单的性能测试
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [6]:
#定义一个对训练、测试数据进行标准化的处理函数
def standard_scale(X_train,X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train,X_test

In [17]:
#定义一个获取随机数据的函数
def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]

In [8]:
#对数据集进行标
X_train,X_test = standard_scale(mnist.train.images,mnist.test.images)

In [9]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [15]:
#创建一个AGN自编码器的实例，输入节点数为784，同时将噪声系数scale设为0.01
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,
                                              n_hidden = 200,
                                              transfer_function = tf.nn.softplus,
                                              optimizer = tf.train.AdamOptimizer(learning_rate=0.001),
                                              scale = 0.01)

In [18]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train,batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples*batch_size
    if epoch % display_step == 0:
        print('Epoch:','%04d'%(epoch+1),'cost=','{:.9f}'.format(avg_cost))

Epoch: 0001 cost= 45445.916527273
Epoch: 0002 cost= 42612.156227273
Epoch: 0003 cost= 43387.770254545
Epoch: 0004 cost= 49525.550863636
Epoch: 0005 cost= 83516.518463636
Epoch: 0006 cost= 70763.390345455
Epoch: 0007 cost= 49966.543963636
Epoch: 0008 cost= 67485.044136364
Epoch: 0009 cost= 60004.245690909
Epoch: 0010 cost= 46704.477836364
Epoch: 0011 cost= 45677.509395455
Epoch: 0012 cost= 47292.632931818
Epoch: 0013 cost= 45479.448477273
Epoch: 0014 cost= 51461.811440909
Epoch: 0015 cost= 46911.890168182
Epoch: 0016 cost= 46454.842904545
Epoch: 0017 cost= 44237.087450000
Epoch: 0018 cost= 42088.748681818
Epoch: 0019 cost= 43853.946654545
Epoch: 0020 cost= 46879.267472727


In [19]:
print("Total cost:"+str(autoencoder.calc_total_cost(X_test)))

Total cost:3910121.8


In [ ]:
#实现自编码器和实现一个但隐含层的神经网络差不多，只不过是数据输入时做了标准化
#并加上了一个高斯噪声，同时我们的输出结果不是数字分类结果，而是复原的数据